In [10]:
import openmm.app as app
import openmm as mm
import openmm.unit as unit
import numpy as np
import jax
import jax.numpy as jnp
import dmff
from dmff.api.xmlio import XMLIO
from dmff.api.paramset import ParamSet
from dmff.generators.classical import CoulombGenerator, LennardJonesGenerator
from dmff.api.hamiltonian import Hamiltonian
from dmff.operators import ParmedLennardJonesOperator
from dmff import NeighborListFreud
from dmff.mbar import ReweightEstimator
import mdtraj as md
from tqdm import tqdm, trange
import parmed
import sys
import os

In [76]:
ReweightEstimator??

Init signature:
ReweightEstimator(
    ref_energies,
    base_energies=None,
    volume=None,
    temperature=300.0,
    pressure=1.0,
)
Docstring:      <no docstring>
Source:        
class ReweightEstimator:
    def __init__(
        self,
        ref_energies,
        base_energies=None,
        volume=None,
        temperature=300.0,
        pressure=1.0,
    ):
        self.beta = 1.0 / temperature / 8.314 * 1000.0
        self.ref_energies = jnp.array(ref_energies)
        if base_energies is None:
            self.base_energies = jnp.zeros(ref_energies.shape)
        else:
            self.base_energies = jnp.array(base_energies)
        if volume is not None:
            self.pv = jnp.array(volume * pressure * 0.06023)
        else:
            self.pv = jnp.zeros(ref_energies.shape)

    def estimate_weight(self, uinit):
        unew = (uinit + self.base_energies + self.pv) * self.beta
        uref = (self.ref_energies + self.pv) * self.beta
        deltaU = unew - uref
       

In [14]:
from ase.io import read, write
atoms = read("/home/yutao/softwares/RASPA2/../MIL-120/MIL-120.cif")
#write("/home/yutao/softwares/RASPA2/../MIL-120/MIL-120.xyz",atoms,write_charges=True)

In [16]:
atoms.get_inital_charges

AttributeError: 'Atoms' object has no attribute 'get_inital_charges'

In [17]:
from ase.io import read
import ase.io.cif as cif

# Load the CIF file
atoms = read("/home/yutao/softwares/RASPA2/../MIL-120/MIL-120.cif")

# Open the CIF file for parsing
cif_file = cif.read("/home/yutao/softwares/RASPA2/../MIL-120/MIL-120.cif")

# Extract charge information from the CIF data
charges = cif_file['charge']  # Use the appropriate key from your CIF file

# Check the extracted charges
print(charges)


AttributeError: module 'ase.io.cif' has no attribute 'read'

In [19]:
import openbabel

In [20]:
# Input CIF file and output PDB file names
input_cif_file = "/home/yutao/softwares/RASPA2/../MIL-120/MIL-120.cif"
output_pdb_file = "/home/yutao/softwares/RASPA2/../MIL-120/MIL-120.pdb"

# Initialize the Open Babel converter
conv = openbabel.OBConversion()

# Set input and output formats
conv.SetInAndOutFormats("cif", "pdb")

# Create the input molecule
mol = openbabel.OBMol()

# Read the CIF file
conv.ReadFile(mol, input_cif_file)

# Create the output file
conv.WriteFile(mol, output_pdb_file)

# Clean up
conv.CloseOutFile()


/home/yutao/anaconda3/envs/dmff/lib/python3.11/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')


In [21]:
# Input CIF file and output PDB file names
input_cif_file = "/home/yutao/softwares/RASPA2/../MIL-120/MIL-120.cif"
output_pdb_file = "/home/yutao/softwares/RASPA2/../MIL-120/MIL-120.mol2"

# Initialize the Open Babel converter
conv = openbabel.OBConversion()

# Set input and output formats
conv.SetInAndOutFormats("cif", "mol2")

# Create the input molecule
mol = openbabel.OBMol()

# Read the CIF file
conv.ReadFile(mol, input_cif_file)

# Create the output file
conv.WriteFile(mol, output_pdb_file)

# Clean up
conv.CloseOutFile()


In [ ]:
from ase import Atoms

# Create an ASE Atoms object with atomic coordinates and charges
# Replace these coordinates and charges with your actual data


atoms = read("/home/yutao/softwares/RASPA2/../MIL-120/MIL-120.cif")

# Open an XYZ file for writing
with open("/home/yutao/softwares/RASPA2/../MIL-120/MIL-120.xyz", 'w') as f:
    num_atoms = len(atoms)
    
    # Write the number of atoms in the first line
    f.write(f"{num_atoms}\n")
    
    # Write a comment line (can be blank)
    f.write("XYZ file with charges\n")
    
    # Write atomic coordinates and charges
    for i in range(num_atoms):
        symbol = atoms.get_chemical_symbols()[i]
        position = atoms.get_positions()[i]
        charge = charges[i]
        f.write(f"{symbol} {position[0]:.6f} {position[1]:.6f} {position[2]:.6f} {charge:.6f}\n")


In [23]:
prm_top = parmed.load_file("/home/yutao/softwares/RASPA2/../MIL-120/MIL-120.pdb")

In [29]:
prm_top = parmed.load_file("Lig.top")
prm_top_500 = prm_top * 500
dmfftop = dmff.DMFFTopology(from_top=prm_top_500.topology)
prmop = ParmedLennardJonesOperator()
dmfftop = prmop(dmfftop, gmx_top = prm_top_500)

prmop.renderLennardJonesXML("init.xml")
xmlio = XMLIO()
xmlio.loadXML("init.xml")
ffinfo = xmlio.parseXML()
cov_mat = dmfftop.buildCovMat()

paramset = ParamSet()
lj_gen = LennardJonesGenerator(ffinfo, paramset)
col_gen = CoulombGenerator(ffinfo, paramset)

KeyError: 'CoulombForce'

In [56]:
top = app.GromacsTopFile("Lig_500.top")
print(type(top))

<class 'openmm.app.gromacstopfile.GromacsTopFile'>


In [82]:
pdb = app.PDBFile("init.pdb")
top = app.GromacsTopFile("Lig_500.top")
top.topology.setPeriodicBoxVectors(pdb.topology.getPeriodicBoxVectors())
system = top.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds, hydrogenMass=3*unit.dalton)

In [87]:
# Assuming you have a CustomNonbondedForce in the system
for force_index in range(system.getNumForces()):
    force = system.getForce(force_index)
    if isinstance(force, mm.NonbondedForce):
        for particle_index in range(force.getNumParticles()):
            parameters = force.getParticleParameters(particle_index)
            print(parameters)
            mass = system.getParticleMass(particle_index)
            charge = parameters[0]  # Assuming charge is the first parameter
            print(f"Particle {particle_index + 1}: Mass = {mass}, Charge = {charge}")

[Quantity(value=-0.172, unit=elementary charge), Quantity(value=0.339967, unit=nanometer), Quantity(value=0.359824, unit=kilojoule/mole)]
Particle 1: Mass = 10.0 Da, Charge = -0.172 e
[Quantity(value=0.1245, unit=elementary charge), Quantity(value=0.259964, unit=nanometer), Quantity(value=0.06276, unit=kilojoule/mole)]
Particle 2: Mass = 3.0 Da, Charge = 0.1245 e
[Quantity(value=-0.1382, unit=elementary charge), Quantity(value=0.339967, unit=nanometer), Quantity(value=0.359824, unit=kilojoule/mole)]
Particle 3: Mass = 10.0 Da, Charge = -0.1382 e
[Quantity(value=0.1449, unit=elementary charge), Quantity(value=0.259964, unit=nanometer), Quantity(value=0.06276, unit=kilojoule/mole)]
Particle 4: Mass = 3.0 Da, Charge = 0.1449 e
[Quantity(value=-0.1802, unit=elementary charge), Quantity(value=0.339967, unit=nanometer), Quantity(value=0.359824, unit=kilojoule/mole)]
Particle 5: Mass = 10.0 Da, Charge = -0.1802 e
[Quantity(value=0.1457, unit=elementary charge), Quantity(value=0.259964, unit=n

In [86]:
# Assuming you have a CustomNonbondedForce in the system
for force_index in range(system.getNumForces()):
    force = system.getForce(force_index)
    if isinstance(force, mm.NonbondedForce):
        print(type(force))
        for particle_index in range(force.getNumParticles()):
            parameters = force.getParticleParameters(particle_index)
            mass = system.getParticleMass(particle_index)
            charge = parameters[0]  # Assuming charge is the first parameter
            print(f"Particle {particle_index + 1}: Mass = {mass}, Charge = {charge}")


<class 'openmm.openmm.NonbondedForce'>
Particle 1: Mass = 10.0 Da, Charge = -0.172 e
Particle 2: Mass = 3.0 Da, Charge = 0.1245 e
Particle 3: Mass = 10.0 Da, Charge = -0.1382 e
Particle 4: Mass = 3.0 Da, Charge = 0.1449 e
Particle 5: Mass = 10.0 Da, Charge = -0.1802 e
Particle 6: Mass = 3.0 Da, Charge = 0.1457 e
Particle 7: Mass = 10.0 Da, Charge = -0.1382 e
Particle 8: Mass = 3.0 Da, Charge = 0.1449 e
Particle 9: Mass = 10.0 Da, Charge = -0.172 e
Particle 10: Mass = 3.0 Da, Charge = 0.1245 e
Particle 11: Mass = 12.0 Da, Charge = 0.0622 e
Particle 12: Mass = 8.0 Da, Charge = 0.314 e
Particle 13: Mass = 3.0 Da, Charge = -0.0216 e
Particle 14: Mass = 3.0 Da, Charge = -0.0216 e
Particle 15: Mass = 14.0 Da, Charge = -0.6083 e
Particle 16: Mass = 3.0 Da, Charge = 0.3914 e
Particle 17: Mass = 10.0 Da, Charge = -0.172 e
Particle 18: Mass = 3.0 Da, Charge = 0.1245 e
Particle 19: Mass = 10.0 Da, Charge = -0.1382 e
Particle 20: Mass = 3.0 Da, Charge = 0.1449 e
Particle 21: Mass = 10.0 Da, Charge

In [60]:
top.createSystem??

Signature:
top.createSystem(
    nonbondedMethod=NoCutoff,
    nonbondedCutoff=Quantity(value=1.0, unit=nanometer),
    constraints=None,
    rigidWater=True,
    implicitSolvent=None,
    soluteDielectric=1.0,
    solventDielectric=78.5,
    ewaldErrorTolerance=0.0005,
    removeCMMotion=True,
    hydrogenMass=None,
    switchDistance=None,
)
Source:   
    def createSystem(self, nonbondedMethod=ff.NoCutoff, nonbondedCutoff=1.0*unit.nanometer,
                     constraints=None, rigidWater=True, implicitSolvent=None, soluteDielectric=1.0, solventDielectric=78.5,
                     ewaldErrorTolerance=0.0005, removeCMMotion=True, hydrogenMass=None, switchDistance=None):
        """Construct an OpenMM System representing the topology described by this
        top file.

        Parameters
        ----------
        nonbondedMethod : object=NoCutoff
            The method to use for nonbonded interactions.  Allowed values are
            NoCutoff, CutoffNonPeriodic, CutoffPeriodic, 

In [32]:
xmlio.loadXML("../../tests/data/water4.xml")
ffinfo = xmlio.parseXML()
ffinfo

{'Operators': {},
 'AtomTypes': [{'name': 'spce-O',
   'class': 'OW',
   'element': 'O',
   'mass': '15.99943'},
  {'name': 'spce-H', 'class': 'HW', 'element': 'H', 'mass': '1.007947'}],
 'Residues': [{'name': 'HOH',
   'particles': [{'name': 'O', 'type': 'spce-O', 'charge': -0.82},
    {'name': 'H1', 'type': 'spce-H', 'charge': 0.41},
    {'name': 'H2', 'type': 'spce-H', 'charge': 0.41}],
   'bonds': [{'atomName1': 'O', 'atomName2': 'H1'},
    {'atomName1': 'O', 'atomName2': 'H2'}],
   'externals': [],
   'vsites': []}],
 'Forces': {'LennardJonesForce': {'meta': {'lj14scale': '0.50000'},
   'node': [{'name': 'Atom',
     'attrib': {'epsilon': '0.359824',
      'sigma': '0.3399670000000001',
      'type': 'ca'}},
    {'name': 'Atom',
     'attrib': {'epsilon': '0.06276',
      'sigma': '0.25996400000000003',
      'type': 'ha'}},
    {'name': 'Atom',
     'attrib': {'epsilon': '0.45773',
      'sigma': '0.3399670000000001',
      'type': 'c3'}},
    {'name': 'Atom',
     'attrib': {'ep

In [33]:
generator = CoulombGenerator(ffinfo, paramset)

In [7]:
prm_top.atoms

AtomList([
	<Atom C1 [0]; In MOL 0>
	<Atom H1 [1]; In MOL 0>
	<Atom C2 [2]; In MOL 0>
	<Atom H2 [3]; In MOL 0>
	<Atom C3 [4]; In MOL 0>
	<Atom H3 [5]; In MOL 0>
	<Atom C4 [6]; In MOL 0>
	<Atom H4 [7]; In MOL 0>
	<Atom C5 [8]; In MOL 0>
	<Atom H5 [9]; In MOL 0>
	<Atom C6 [10]; In MOL 0>
	<Atom C7 [11]; In MOL 0>
	<Atom H6 [12]; In MOL 0>
	<Atom H7 [13]; In MOL 0>
	<Atom O1 [14]; In MOL 0>
	<Atom H8 [15]; In MOL 0>
])

In [5]:
lj_force = lj_gen.createPotential(
    dmfftop, nonbondedMethod=app.CutoffPeriodic, nonbondedCutoff=1.0, args={})

### 基于给定拓扑进行采样的函数

In [36]:
def runMD(topfile, pdbfile, trajfile, length):
    try:
        os.remove("Lig_500.top")
    except:
        pass
    top_prm = parmed.load_file(topfile)
    top_500 = top_prm * 500
    top_500.save("Lig_500.top")
    pdb = app.PDBFile(pdbfile)
    top = app.GromacsTopFile("Lig_500.top")
    top.topology.setPeriodicBoxVectors(pdb.topology.getPeriodicBoxVectors())
    system = top.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds, hydrogenMass=3*unit.dalton)
    for force in system.getForces():
        if isinstance(force, mm.NonbondedForce):
            force.setUseDispersionCorrection(False)
    system.addForce(mm.MonteCarloBarostat(1.0*unit.bar, 300*unit.kelvin, 25))
    integ = mm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 2.5*unit.femtosecond)
    simulation = app.Simulation(top.topology, system, integ)
    simulation.reporters.append(app.StateDataReporter(sys.stdout, 400, time=True, potentialEnergy=True, temperature=True, density=True, speed=True, remainingTime=True, totalSteps=int(length) * 400))
    simulation.reporters.append(app.DCDReporter(trajfile, 400))
    simulation.context.setPositions(pdb.getPositions())
    simulation.minimizeEnergy(maxIterations=200)
    simulation.step(int(length) * 400)
    os.remove("Lig_500.top")

# runMD("Lig.top", "init.pdb", "traj.dcd", 100)

In [37]:
def rerun_energy(pdb, traj, top, skip=20, removeLJ=True, skpi=0):
    samples = md.load(traj, top=pdb)[skip:]
    try:
        os.remove("Lig_500.top")
    except:
        pass
    top_prm = parmed.load_file(top)
    top_500 = top_prm * 500
    top_500.save("Lig_500.top")
    pdb = app.PDBFile(pdb)
    top = app.GromacsTopFile("Lig_500.top")
    os.remove("Lig_500.top")
    top.topology.setPeriodicBoxVectors(pdb.topology.getPeriodicBoxVectors())
    system = top.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds, hydrogenMass=3*unit.dalton)
    
    for force in system.getForces():
        if isinstance(force, mm.NonbondedForce):
            force.setUseDispersionCorrection(False)
            if removeLJ:
                for npart in range(force.getNumParticles()):
                    chrg, sig, eps = force.getParticleParameters(npart)
                    force.setParticleParameters(npart, chrg, 1.0, 0.0)
                for nex in range(force.getNumExceptions()):
                    p1, p2, chrg, sig, eps = force.getExceptionParameters(nex)
                    force.setExceptionParameters(nex, p1, p2, chrg, 1.0, 0.0)
    integ = mm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 2.5*unit.femtosecond)
    ctx = mm.Context(system, integ)
    energies = []
    for frame in tqdm(samples):
        ctx.setPositions(frame.xyz[0] * unit.nanometer)
        ctx.setPeriodicBoxVectors(*frame.unitcell_vectors[0])
        ctx.applyConstraints(1e-10)
        state = ctx.getState(getEnergy=True)
        energy = state.getPotentialEnergy().value_in_unit(unit.kilojoule_per_mole)
        energies.append(energy)
    return np.array(energies)

# rerun_energy("init.pdb", "traj.dcd", "Lig.top")

In [38]:
def rerun_dmff_lennard_jones(params, pdb, traj, efunc, skip=0):
    samples = md.load(traj, top=pdb)[skip:]
    energies = []
    nblist = NeighborListFreud(samples.unitcell_vectors[0], 1.0, cov_mat)
    xyzs_jnp = jnp.array(samples.xyz)
    cell_jnp = jnp.array(samples.unitcell_vectors)
    energies = []
    nblist = NeighborListFreud(samples.unitcell_vectors[0], 1.0, cov_mat)
    xyzs_jnp = jnp.array(samples.xyz)
    cell_jnp = jnp.array(samples.unitcell_vectors)
    energies = []
    for nframe in trange(len(samples)):
        frame = samples[nframe]
        # calc pair
        pairs = jnp.array(nblist.allocate(frame.xyz[0], frame.unitcell_vectors[0]))
        ener = efunc(xyzs_jnp[nframe,:,:], cell_jnp[nframe,:,:], pairs, params)
        energies.append(ener.reshape((1,)))
    energies = jnp.concatenate(energies)
    return energies

In [52]:
import optax
optimizer = optax.adam(0.001)
opt_state = optimizer.init(paramset)

In [48]:
print(paramset.to_jax())

None


In [46]:
lbfgs = None

os.system("cp Lig.top loop-0.top")
for nloop in range(1, 51):
    # sample
    print("SAMPLE")
    runMD(f"loop-{nloop-1}.top", "init.pdb", f"loop-{nloop}.dcd", length=70)
    print("RERUN")
    ener = rerun_energy("init.pdb", f"loop-{nloop}.dcd", f"loop-{nloop-1}.top", removeLJ=False, skip=20)
    ener_no_lj = rerun_energy("init.pdb", f"loop-{nloop}.dcd", f"loop-{nloop-1}.top", skip=20)
    print("ESTIMATOR")
    traj = md.load(f"loop-{nloop}.dcd", top="init.pdb")[20:]
    estimator = ReweightEstimator(ener, base_energies=ener_no_lj, volume=traj.unitcell_volumes)

    print("CALC DENSE")
    density = md.density(traj) * 0.001

    # get loss & grad
    def loss(paramset):
        lj_jax = rerun_dmff_lennard_jones(paramset, "init.pdb", f"loop-{nloop}.dcd", lj_force, skip=20)
        weight = estimator.estimate_weight(lj_jax)
        dens = weight * density
        dens = dens.mean()
        return jnp.power(dens - 0.85, 2)
    
    v_and_g = jax.value_and_grad(loss, 0)
    v, g = v_and_g(paramset)
    print("Loss:", v)
    # update parameters
    updates, opt_state = optimizer.update(g, opt_state)
    paramset = optax.apply_updates(paramset, updates)
    paramset = jax.tree_map(lambda x: jnp.clip(x, 0.0, 1e8), paramset)
    
    # upate ffinfo
    lj_gen.overwrite(paramset)
    prmop.overwriteLennardJones(prm_top, lj_gen.ffinfo)
    prm_top.save(f"loop-{nloop}.top")

SAMPLE


KeyboardInterrupt: 

This part is to calculate evaporation enthalpy

In [61]:
import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]=".95"
import openmm.app as app
import openmm as mm
import openmm.unit as unit
import numpy as np
import jax
import jax.numpy as jnp
import dmff
from dmff.api.xmlio import XMLIO
from dmff.api.paramset import ParamSet
from dmff.generators.classical import CoulombGenerator, LennardJonesGenerator
from dmff.api.hamiltonian import Hamiltonian
from dmff.operators import ParmedLennardJonesOperator
from dmff import NeighborListFreud
from dmff.mbar import ReweightEstimator
import mdtraj as md
from tqdm import tqdm, trange
import parmed
import sys

In [62]:
os.system("cp LIQ_T297.15.pdb liquid.pdb")
try:
    os.remove("GMX.top")
except:
    pass
os.system("cp Lig.top GMX.top")

0

In [63]:
particle_number =  500             #液体pdb文件中的分子数
target_han= 50.52                  #拟合的目标蒸发焓
SET_temperature=  293.15           #温度设定
time_gap=   2                      #分子动力学模拟过程中每一个frame的时间间隔，单位是皮秒picosecond   推荐2-4ps
loop_time =   50                   #迭代循环次数    推荐50-100
save_step =   125                  #保存的frame数  推荐150-200
skip_step =    25                   #计算各种物理常数之前，达到平衡所需要的步数
length_step = (skip_step+save_step) #分子动力学模拟生成的dcd文件存储的frame个数，与time_gap共同决定总模拟的时长，总时长=length_step*time_gap

In [64]:
prm_top = parmed.load_file("GMX.top")
prm_top_particle_number = prm_top * particle_number
dmfftop = dmff.DMFFTopology(from_top=prm_top_particle_number.topology)
prmop = ParmedLennardJonesOperator()
dmfftop = prmop(dmfftop, gmx_top = prm_top_particle_number)

prmop.renderLennardJonesXML("init.xml")
xmlio = XMLIO()
xmlio.loadXML("init.xml")
ffinfo = xmlio.parseXML()
cov_mat = dmfftop.buildCovMat()

In [65]:
paramset = ParamSet()
lj_gen = LennardJonesGenerator(ffinfo, paramset)
lj_force = lj_gen.createPotential(
    dmfftop, nonbondedMethod=app.CutoffPeriodic, nonbondedCutoff=1.0, args={})

In [66]:
gap_step=int(time_gap*400)
def runMD(topfile, pdbfile, trajfile, length):
    try:
        os.remove("Lig_particle_number.top")
    except:
        pass
    top_prm = parmed.load_file(topfile)
    top_500 = top_prm * particle_number
    top_500.save("Lig_particle_number.top")
    pdb = app.PDBFile(pdbfile)
    top = app.GromacsTopFile("Lig_particle_number.top")
    top.topology.setPeriodicBoxVectors(pdb.topology.getPeriodicBoxVectors())
    system = top.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds, hydrogenMass=3*unit.dalton)
    for force in system.getForces():
        if isinstance(force, mm.NonbondedForce):
            force.setUseDispersionCorrection(False)
    system.addForce(mm.MonteCarloBarostat(1.0*unit.bar, SET_temperature*unit.kelvin, 25))
    integ = mm.LangevinIntegrator(SET_temperature*unit.kelvin, 1/unit.picosecond, 2.5*unit.femtosecond)
    simulation = app.Simulation(top.topology, system, integ)
    simulation.reporters.append(app.StateDataReporter(sys.stdout, gap_step, time=True, potentialEnergy=True, temperature=True, density=True, speed=True, remainingTime=True, totalSteps=int(length) * gap_step))
    simulation.reporters.append(app.DCDReporter(trajfile, gap_step))
    simulation.context.setPositions(pdb.getPositions())
    simulation.minimizeEnergy(maxIterations=200)
    simulation.step(int(length) * gap_step)
    os.remove("Lig_particle_number.top")

In [79]:
top

In [67]:
def runMD_gas_NVT(topfile, pdbfile, trajfile, length):
    try:
        os.remove("GAS.top")
    except:
        pass
    top_prm = parmed.load_file(topfile)
    top_GAS = top_prm 
    top_GAS.save("GAS.top")
    pdb = app.PDBFile(pdbfile)
    top = app.GromacsTopFile("GAS.top")
    top.topology.setPeriodicBoxVectors(pdb.topology.getPeriodicBoxVectors())
    system = top.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds, hydrogenMass=3*unit.dalton)
    for force in system.getForces():
        if isinstance(force, mm.NonbondedForce):
            force.setUseDispersionCorrection(False)
    integ = mm.LangevinIntegrator(SET_temperature*unit.kelvin, 1/unit.picosecond, 2.5*unit.femtosecond)
    simulation = app.Simulation(top.topology, system, integ)
    simulation.reporters.append(app.StateDataReporter(sys.stdout, gap_step, time=True, potentialEnergy=True, temperature=True, density=True, speed=True, remainingTime=True, totalSteps=int(length) * gap_step))
    simulation.reporters.append(app.DCDReporter(trajfile, gap_step))
    simulation.context.setPositions(pdb.getPositions())
    simulation.minimizeEnergy(maxIterations=200)
    simulation.step(int(length) * gap_step)
    os.remove("GAS.top")

In [69]:
def rerun_energy(pdb, traj, top, skip=20, removeLJ=True, skpi=0):
    samples = md.load(traj, top=pdb)[skip:]
    try:
        os.remove("Lig_particle_number.top")
    except:
        pass
    top_prm = parmed.load_file(top)
    top_500 = top_prm * particle_number
    top_500.save("Lig_particle_number.top")
    pdb = app.PDBFile(pdb)
    top = app.GromacsTopFile("Lig_particle_number.top")
    os.remove("Lig_particle_number.top")
    top.topology.setPeriodicBoxVectors(pdb.topology.getPeriodicBoxVectors())
    system = top.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds, hydrogenMass=3*unit.dalton)
    
    for force in system.getForces():
        if isinstance(force, mm.NonbondedForce):
            force.setUseDispersionCorrection(False)
            if removeLJ:
                for npart in range(force.getNumParticles()):
                    chrg, sig, eps = force.getParticleParameters(npart)
                    force.setParticleParameters(npart, chrg, 1.0, 0.0)
                for nex in range(force.getNumExceptions()):
                    p1, p2, chrg, sig, eps = force.getExceptionParameters(nex)
                    force.setExceptionParameters(nex, p1, p2, chrg, 1.0, 0.0)
    integ = mm.LangevinIntegrator(SET_temperature*unit.kelvin, 1/unit.picosecond, 2.5*unit.femtosecond)
    ctx = mm.Context(system, integ)
    energies = []
    for frame in tqdm(samples):
        ctx.setPositions(frame.xyz[0] * unit.nanometer)
        ctx.setPeriodicBoxVectors(*frame.unitcell_vectors[0])
        ctx.applyConstraints(1e-10)
        state = ctx.getState(getEnergy=True)
        energy = state.getPotentialEnergy().value_in_unit(unit.kilojoule_per_mole)
        energies.append(energy)
    return np.array(energies)

In [70]:
def rerun_energy_gas(pdb, traj, top, skip=50, removeLJ=True, skpi=0):
    samples = md.load(traj, top=pdb)[skip:]
    try:
        os.remove("GAS.top")
    except:
        pass
    top_prm = parmed.load_file(top)
    top_gas = top_prm
    top_gas.save("GAS.top")
    pdb = app.PDBFile(pdb)
    top = app.GromacsTopFile("GAS.top")
    os.remove("GAS.top")
    top.topology.setPeriodicBoxVectors(pdb.topology.getPeriodicBoxVectors())
    system = top.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds, hydrogenMass=3*unit.dalton)
    
    for force in system.getForces():
        if isinstance(force, mm.NonbondedForce):
            force.setUseDispersionCorrection(False)
            if removeLJ:
                for npart in range(force.getNumParticles()):
                    chrg, sig, eps = force.getParticleParameters(npart)
                    force.setParticleParameters(npart, chrg, 1.0, 0.0)
                for nex in range(force.getNumExceptions()):
                    p1, p2, chrg, sig, eps = force.getExceptionParameters(nex)
                    force.setExceptionParameters(nex, p1, p2, chrg, 1.0, 0.0)
    integ = mm.LangevinIntegrator(SET_temperature*unit.kelvin, 1/unit.picosecond, 2.5*unit.femtosecond)
    ctx = mm.Context(system, integ)
    energies = []
    for frame in tqdm(samples):
        ctx.setPositions(frame.xyz[0] * unit.nanometer)
        ctx.setPeriodicBoxVectors(*frame.unitcell_vectors[0])
        ctx.applyConstraints(1e-10)
        state = ctx.getState(getEnergy=True)
        energy = state.getPotentialEnergy().value_in_unit(unit.kilojoule_per_mole)
        energies.append(energy)
    return np.array(energies)

In [71]:
def rerun_dmff_lennard_jones(params, pdb, traj, efunc, skip=0):
    samples = md.load(traj, top=pdb)[skip:]
    energies = []
    nblist = NeighborListFreud(samples.unitcell_vectors[0], 1.0, cov_mat)
    xyzs_jnp = jnp.array(samples.xyz)
    cell_jnp = jnp.array(samples.unitcell_vectors)
    energies = []
    nblist = NeighborListFreud(samples.unitcell_vectors[0], 1.0, cov_mat)
    xyzs_jnp = jnp.array(samples.xyz)
    cell_jnp = jnp.array(samples.unitcell_vectors)
    energies = []
    for nframe in trange(len(samples)):
        frame = samples[nframe]
        # calc pair
        pairs = jnp.array(nblist.allocate(frame.xyz[0], frame.unitcell_vectors[0]))
        ener = efunc(xyzs_jnp[nframe,:,:], cell_jnp[nframe,:,:], pairs, params)
        energies.append(ener.reshape((1,)))
    energies = jnp.concatenate(energies)
    return energies

In [72]:
import optax
optimizer = optax.adam(0.001)
opt_state = optimizer.init(paramset)

In [73]:
lbfgs = None


os.system("cp GMX.top loop-0.top")
Losslist=[]
enthalpy_of_evaporation_list=[]
energy_gas_list=[]
energy_liquid_list=[]
for nloop in range(1, loop_time+1):
    # sample liquid
    print("SAMPLE_liquid")
    try:
        runMD(f"loop-{nloop-1}.top", "liquid.pdb", f"loop-{nloop}.dcd", length=length_step)
    except:
        runMD(f"loop-{nloop-1}.top", "liquid.pdb", f"loop-{nloop}.dcd", length=length_step)
        
    # sample GAs
    print("SAMPLE_GAS")

    try:
        runMD_gas_NVT(f"loop-{nloop-1}.top", "GAS.pdb", f"loop-{nloop}-gas.dcd", length=skip_step+1000)
    except:
        runMD_gas_NVT(f"loop-{nloop-1}.top", "GAS.pdb", f"loop-{nloop}-gas.dcd", length=skip_step+1000)
        
    #重新计算能量   不可微的计算液态能量 、液态不包含LJ势能的能量 、气态能量
    print("RERUN")
    ener = rerun_energy("liquid.pdb", f"loop-{nloop}.dcd", f"loop-{nloop-1}.top", removeLJ=False, skip=skip_step)
    energy_liquid_list.append(ener.mean())
    ener_gas = rerun_energy_gas("GAS.pdb", f"loop-{nloop}-gas.dcd", f"loop-{nloop-1}.top", removeLJ=False, skip=skip_step).mean()
    energy_gas_list.append(ener_gas)
    ener_no_lj = rerun_energy("liquid.pdb", f"loop-{nloop}.dcd", f"loop-{nloop-1}.top", skip=skip_step)
    
    print("ESTIMATOR")
    traj = md.load(f"loop-{nloop}.dcd", top="liquid.pdb")[skip_step:]
    estimator = ReweightEstimator(ener, base_energies=ener_no_lj, volume=traj.unitcell_volumes)
    
    #计算当前蒸发焓
    print("CALC han")
    ener_lquid_now = (ener / particle_number).mean()
    enthalpy_of_evaporation_now = ener_gas - ener_lquid_now + 8.314 * SET_temperature * 0.001
    enthalpy_of_evaporation_list.append(enthalpy_of_evaporation_now)
    with open('enthalpy_of_evaporation_list.txt', 'a') as f:
        f.write("%s\n" % str(enthalpy_of_evaporation_now))
        
    # get loss & grad
    def loss(paramset):
        lj_jax = rerun_dmff_lennard_jones(paramset, "liquid.pdb", f"loop-{nloop}.dcd", lj_force, skip=skip_step)
        weight = estimator.estimate_weight(lj_jax)
        ener_lquid = (weight * ener) / particle_number 
        ener_lquid = ener_lquid.mean()
        enthalpy_of_evaporation = ener_gas - ener_lquid + 8.314*SET_temperature*0.001

        return  jnp.power(enthalpy_of_evaporation - target_han, 2)
    
    v_and_g = jax.value_and_grad(loss, 0)
    v, g = v_and_g(paramset)
    print("Loss:", v)
    Losslist.append(v)
    Losslist_np =jax.device_get(v)
    
    with open('Losslist.txt', 'a') as f:
        f.write("%s\n" % str(Losslist_np))
        
    # update parameters
    updates, opt_state = optimizer.update(g, opt_state)
    paramset = optax.apply_updates(paramset, updates)
    paramset = jax.tree_map(lambda x: jnp.clip(x, 0.0, 1e8), paramset)
    
    # upate ffinfo
    lj_gen.overwrite(paramset)
    prmop.overwriteLennardJones(prm_top, lj_gen.ffinfo)
    prm_top.save(f"loop-{nloop}.top")



SAMPLE_liquid
#"Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Density (g/mL)","Speed (ns/day)","Time Remaining"
1.9999999999999685,26438.38039277213,258.7586641946508,0.5600297362224183,0,--
3.999999999999926,27094.240007554254,288.941620358525,0.6149986517809132,26.2,16:14
6.000000000000238,26201.310525603447,292.0335702242387,0.6746282115080577,25.8,16:22
8.00000000000055,25242.71316842964,293.0151684132643,0.7388164230728739,25.5,16:29
10.000000000000153,23654.70545834885,295.6339651460507,0.8198055689649877,25.1,16:38
11.999999999999755,22802.8837840601,294.6568543404851,0.8535839276262555,24.8,16:42
13.999999999999357,22449.2874874654,290.53306277458955,0.8946125413330721,24.4,16:51
15.999999999998959,21651.817254958543,298.96809783990915,0.9248711327675437,24.3,16:51
17.999999999999982,22170.173618794044,299.52598317643896,0.9458847144526406,24.1,16:50
20.000000000001005,21035.245643676113,293.9577790487992,0.9601460621496909,23.9,16:50
22.00000000000203,21128.127997

KeyboardInterrupt: 

In [78]:
import jax
import jax.numpy as np

def loss_function(x):
    # Example loss function with a min operation
    return np.min(x**2 + 3*x + 5)

# Computing the gradient of the loss function using JAX
grad_loss = jax.grad(loss_function)

# Example usage in optimization (gradient descent)
learning_rate = 0.01
x_init = np.array([1.0, 2.0, 3.0])  # Initial guess

for _ in range(100):
    gradient = grad_loss(x_init)
    x_init = x_init - learning_rate * gradient

print("Optimal x:", x_init)
print("Minimum loss:", loss_function(x_init))


Optimal x: [-1.16845111  2.          3.        ]
Minimum loss: 2.8599246662857594
